In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [4]:
import malaya

model = malaya.translation.en_ms.transformer(model = 'bigbird')

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

2022-08-01 10:50:48.927201: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-01 10:50:48.947970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 10:50:48.952246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 10:50:48.952904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [5]:
with open('en-ms-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('en-ms-left.test') as fopen:
    left = fopen.read().split('\n')

In [21]:
from tqdm import tqdm

batch_size = 1

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 69744/69744 [6:11:20<00:00,  3.13it/s]


In [27]:
len(results)

69744

In [28]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [29]:
refs = [filtered_right]
sys = filtered_left

In [30]:
r = bleu.corpus_score(sys, refs)

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


In [31]:
r.__dict__

{'name': 'BLEU',
 'score': 59.86353498474623,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '82.2/65.9/54.9/46.4 (BP = 0.982 ratio = 0.982 hyp_len = 2583848 ref_len = 2630014)',
 'bp': 0.9822915210403552,
 'counts': [2123059, 1657649, 1341100, 1102122],
 'totals': [2583848, 2514105, 2444387, 2374856],
 'sys_len': 2583848,
 'ref_len': 2630014,
 'precisions': [82.16655933321155,
  65.9339605943268,
  54.86447113325345,
  46.40795062942764],
 'prec_str': '82.2/65.9/54.9/46.4',
 'ratio': 0.9824464812734837}